# Sélection Manuelle de Régions pour Correction de Ligne de Base

Ce notebook démontre les nouvelles fonctionnalités de sélection manuelle et interactive de régions pour la correction de ligne de base EPR.

## Nouvelles fonctionnalités:
- **Sélection interactive** de régions avec la souris
- **Régions manuelles** définies programmatiquement
- **Mode exclusion** - exclure certaines régions du fit
- **Mode inclusion** - utiliser seulement certaines régions
- **Sélecteur graphique** intégré avec matplotlib

Perfect pour les spectres complexes avec plusieurs signaux ou lignes de base non-uniformes!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import epyr
from epyr.baseline_simple import RegionSelector, create_region_mask_1d, create_region_mask_2d

print(f"EPyR Tools version: {epyr.__version__}")
print("✅ Nouvelles fonctionnalités importées:")
print(f"  RegionSelector: {RegionSelector is not None}")
print(f"  baseline avec régions manuelles: disponible")

## 1. Démonstration Sélection Manuelle 1D

Spectres EPR 1D avec régions définies manuellement.

In [ ]:
# Créer un spectre EPR complexe avec plusieurs signaux
print("🔬 Génération d'un spectre EPR complexe...")

# Axe du champ magnétique
field = np.linspace(3250, 3450, 1200)

# Ligne de base avec courbure
baseline_1d = 25 + 0.08 * (field - 3350) + 0.0002 * (field - 3350)**2

# Signaux EPR multiples
signal1 = 40 * 5**2 / ((field - 3300)**2 + 5**2)  # Signal à 3300 G
signal2 = 60 * 8**2 / ((field - 3350)**2 + 8**2)  # Signal principal à 3350 G  
signal3 = 30 * 6**2 / ((field - 3400)**2 + 6**2)  # Signal à 3400 G
signal4 = 20 * 4**2 / ((field - 3320)**2 + 4**2)  # Signal satellite à 3320 G

total_signal = signal1 + signal2 + signal3 + signal4

# Données finales avec bruit
noise = np.random.normal(0, 1.2, len(field))
data_complex = baseline_1d + total_signal + noise

print(f"✅ Spectre complexe généré: {len(data_complex)} points")
print(f"📊 Signaux aux champs: 3300G, 3320G, 3350G, 3400G")

# Affichage du spectre
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(field, data_complex, 'b-', alpha=0.8, linewidth=1.2, label='Spectre avec ligne de base')
plt.plot(field, baseline_1d, 'r--', alpha=0.7, linewidth=2, label='Ligne de base vraie')
plt.plot(field, total_signal, 'g--', alpha=0.6, linewidth=1.5, label='Signaux EPR')
plt.xlabel('Champ Magnétique (G)')
plt.ylabel('Intensité (a.u.)')
plt.title('Spectre EPR Complexe - Plusieurs Signaux')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(field, signal1, label='Signal 1 (3300G)', alpha=0.8)
plt.plot(field, signal2, label='Signal 2 (3350G)', alpha=0.8)
plt.plot(field, signal3, label='Signal 3 (3400G)', alpha=0.8)
plt.plot(field, signal4, label='Signal 4 (3320G)', alpha=0.8)
plt.xlabel('Champ Magnétique (G)')
plt.ylabel('Intensité (a.u.)')
plt.title('Composantes Individuelles des Signaux')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Définition manuelle de régions à exclure (où sont les signaux)
print("✂️ Définition de régions à exclure manuellement...")

# Régions à exclure : autour de chaque signal
exclude_regions = [
    (3290, 3310),  # Autour du signal à 3300G
    (3315, 3325),  # Autour du signal à 3320G  
    (3340, 3360),  # Autour du signal principal à 3350G
    (3390, 3410),  # Autour du signal à 3400G
]

print(f"📋 Régions définies: {len(exclude_regions)}")
for i, (x1, x2) in enumerate(exclude_regions):
    print(f"   Région {i+1}: {x1}G - {x2}G")

# Correction avec régions manuelles
print("\n🔧 Correction avec exclusion manuelle...")
corrected_manual, baseline_manual = epyr.baseline.baseline_polynomial_1d_simple(
    field, data_complex, {},
    order=2,
    exclude_center=False,      # Désactiver exclusion automatique
    manual_regions=exclude_regions,  # Utiliser régions manuelles
    region_mode='exclude'      # Mode exclusion
)

print("✅ Correction manuelle terminée!")

# Comparaison avec correction automatique (center only)
print("🤖 Correction automatique pour comparaison...")
corrected_auto, baseline_auto = epyr.baseline.baseline_polynomial_1d_simple(
    field, data_complex, {},
    order=2,
    exclude_center=True,       # Exclusion centrale uniquement
    center_fraction=0.3
)

# Visualisation des résultats
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Graphique 1: Données et lignes de base
axes[0,0].plot(field, data_complex, 'b-', alpha=0.7, label='Données originales')
axes[0,0].plot(field, baseline_manual, 'r-', linewidth=2, label='Baseline manuelle')
axes[0,0].plot(field, baseline_auto, 'g--', linewidth=2, label='Baseline auto')
axes[0,0].plot(field, baseline_1d, 'm:', linewidth=2, alpha=0.8, label='Baseline vraie')

# Marquer les régions exclues
for x1, x2 in exclude_regions:
    axes[0,0].axvspan(x1, x2, alpha=0.2, color='red', label='Exclu (manuel)' if x1 == exclude_regions[0][0] else "")

axes[0,0].set_title('Lignes de Base: Manuelle vs Automatique')
axes[0,0].set_xlabel('Champ Magnétique (G)')
axes[0,0].set_ylabel('Intensité (a.u.)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Graphique 2: Signaux corrigés
axes[0,1].plot(field, corrected_manual, 'r-', linewidth=1.5, label='Corrigé (manuel)')
axes[0,1].plot(field, corrected_auto, 'g--', linewidth=1.5, label='Corrigé (auto)')
axes[0,1].plot(field, total_signal, 'b:', linewidth=2, alpha=0.8, label='Signal vrai')
axes[0,1].set_title('Signaux Corrigés')
axes[0,1].set_xlabel('Champ Magnétique (G)')
axes[0,1].set_ylabel('Intensité (a.u.)')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Graphique 3: Erreur correction manuelle
error_manual = corrected_manual - total_signal
axes[1,0].plot(field, error_manual, 'r-', alpha=0.8)
axes[1,0].set_title(f'Erreur Correction Manuelle\n(RMS: {np.sqrt(np.mean(error_manual**2)):.3f})')
axes[1,0].set_xlabel('Champ Magnétique (G)')
axes[1,0].set_ylabel('Erreur (a.u.)')
axes[1,0].grid(True, alpha=0.3)
axes[1,0].axhline(0, color='black', linestyle='--', alpha=0.5)

# Graphique 4: Erreur correction automatique
error_auto = corrected_auto - total_signal
axes[1,1].plot(field, error_auto, 'g-', alpha=0.8)
axes[1,1].set_title(f'Erreur Correction Automatique\n(RMS: {np.sqrt(np.mean(error_auto**2)):.3f})')
axes[1,1].set_xlabel('Champ Magnétique (G)')
axes[1,1].set_ylabel('Erreur (a.u.)')
axes[1,1].grid(True, alpha=0.3)
axes[1,1].axhline(0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print(f"📊 RMS Erreur Manuelle: {np.sqrt(np.mean(error_manual**2)):.4f}")
print(f"📊 RMS Erreur Automatique: {np.sqrt(np.mean(error_auto**2)):.4f}")
improvement = np.sqrt(np.mean(error_auto**2)) / np.sqrt(np.mean(error_manual**2))
print(f"🎯 Amélioration: {improvement:.1f}x meilleure avec sélection manuelle!")

## 2. Sélection Interactive 1D

**Note**: La sélection interactive nécessite un environnement graphique. Dans certains environnements Jupyter, cela pourrait ne pas fonctionner parfaitement.

**Instructions**: 
1. Exécutez la cellule suivante
2. Une fenêtre matplotlib s'ouvrira
3. Cliquez et glissez pour sélectionner les régions des signaux
4. Fermez la fenêtre quand terminé
5. Les régions sélectionnées seront utilisées pour la correction

In [ ]:
# Démonstration sélection interactive (optionnel - nécessite interface graphique)
print("🖱️ Démonstration sélection interactive...")
print("⚠️  Note: Nécessite un environnement graphique interactif")

# Test non-interactif d'abord
try:
    # Simuler une sélection interactive avec des régions pré-définies
    print("\n📋 Simulation de sélection interactive...")
    simulated_regions = [
        (3295, 3305),  # Région du premier signal
        (3345, 3355),  # Région du signal principal
        (3395, 3405),  # Région du dernier signal
    ]
    
    print(f"✅ Régions 'sélectionnées': {len(simulated_regions)}")
    for i, (x1, x2) in enumerate(simulated_regions):
        print(f"   Région {i+1}: {x1:.0f}G - {x2:.0f}G")
    
    # Correction avec sélection simulée
    corrected_interactive, baseline_interactive = epyr.baseline.baseline_polynomial_1d_simple(
        field, data_complex, {},
        order=2,
        manual_regions=simulated_regions,
        region_mode='exclude'
    )
    
    # Affichage résultat
    plt.figure(figsize=(14, 6))
    plt.plot(field, data_complex, 'b-', alpha=0.7, label='Données originales')
    plt.plot(field, baseline_interactive, 'r-', linewidth=2, label='Baseline (sélection simulée)')
    plt.plot(field, corrected_interactive, 'g-', alpha=0.8, label='Données corrigées')
    
    # Marquer régions sélectionnées
    for i, (x1, x2) in enumerate(simulated_regions):
        plt.axvspan(x1, x2, alpha=0.3, color='red', 
                   label='Région exclue' if i == 0 else "")
    
    plt.xlabel('Champ Magnétique (G)')
    plt.ylabel('Intensité (a.u.)')
    plt.title('Résultat avec Sélection Interactive (Simulée)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("✅ Démonstration de sélection interactive réussie!")
    
except Exception as e:
    print(f"⚠️ Sélection interactive non disponible: {e}")
    print("   (Normal dans certains environnements Jupyter)")

# Instructions pour utilisation réelle
print("\n" + "="*60)
print("💡 Pour utilisation interactive réelle:")
print("")
print("corrected, baseline = epyr.baseline.baseline_polynomial_1d_simple(")
print("    x, y, params,")
print("    interactive=True,      # Active la sélection interactive")
print("    region_mode='exclude'  # ou 'include'")
print(")")
print("")
print("📝 Instructions:")
print("  1. Une fenêtre s'ouvre avec le spectre")
print("  2. Cliquez et glissez pour sélectionner régions")
print("  3. Fermez la fenêtre pour continuer")
print("  4. La correction utilise vos sélections")

## 3. Mode Inclusion - Utiliser Seulement Certaines Régions

Parfois il est plus facile de sélectionner les régions de ligne de base plutôt que d'exclure les signaux.

In [ ]:
# Mode inclusion : utiliser seulement certaines régions pour le fit
print("📍 Démonstration mode INCLUSION...")
print("   (utiliser seulement les régions spécifiées)")

# Définir régions de ligne de base pure (sans signaux)
baseline_regions = [
    (3250, 3280),  # Début du spectre
    (3330, 3335),  # Entre signaux
    (3365, 3385),  # Entre signaux
    (3420, 3450),  # Fin du spectre
]

print(f"📋 Régions de ligne de base définies: {len(baseline_regions)}")
for i, (x1, x2) in enumerate(baseline_regions):
    print(f"   Région {i+1}: {x1}G - {x2}G")

# Correction en mode inclusion
print("\n🔧 Correction en mode INCLUSION...")
corrected_include, baseline_include = epyr.baseline.baseline_polynomial_1d_simple(
    field, data_complex, {},
    order=2,
    exclude_center=False,            # Pas d'exclusion automatique
    manual_regions=baseline_regions, # Régions de baseline
    region_mode='include'            # Mode inclusion
)

print("✅ Correction en mode inclusion terminée!")

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Graphique 1: Données et régions utilisées
axes[0].plot(field, data_complex, 'b-', alpha=0.7, label='Données originales')
axes[0].plot(field, baseline_include, 'r-', linewidth=2, label='Baseline (inclusion)')
axes[0].plot(field, baseline_1d, 'g--', linewidth=2, alpha=0.8, label='Baseline vraie')

# Marquer régions incluses
for i, (x1, x2) in enumerate(baseline_regions):
    axes[0].axvspan(x1, x2, alpha=0.3, color='green', 
                   label='Région incluse' if i == 0 else "")

# Créer un masque pour montrer les points utilisés
mask_include = create_region_mask_1d(field, baseline_regions, mode='include')
axes[0].scatter(field[mask_include], data_complex[mask_include], 
               c='red', s=2, alpha=0.6, label='Points utilisés pour fit')

axes[0].set_title('Mode Inclusion - Utiliser Seulement Certaines Régions')
axes[0].set_xlabel('Champ Magnétique (G)')
axes[0].set_ylabel('Intensité (a.u.)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graphique 2: Résultat de la correction
axes[1].plot(field, corrected_include, 'r-', linewidth=1.5, label='Données corrigées (inclusion)')
axes[1].plot(field, total_signal, 'b--', linewidth=2, alpha=0.8, label='Signal vrai')
axes[1].axhline(0, color='black', linestyle=':', alpha=0.5)

axes[1].set_title('Signal Corrigé avec Mode Inclusion')
axes[1].set_xlabel('Champ Magnétique (G)')
axes[1].set_ylabel('Intensité (a.u.)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistiques
error_include = corrected_include - total_signal
rms_include = np.sqrt(np.mean(error_include**2))
print(f"\n📊 Statistiques mode inclusion:")
print(f"   RMS erreur: {rms_include:.4f}")
print(f"   Points utilisés: {np.sum(mask_include)}/{len(field)} ({100*np.sum(mask_include)/len(field):.1f}%)")

## 4. Démonstration 2D - Sélection de Régions

Application aux données EPR 2D avec sélection de régions rectangulaires.

In [ ]:
# Créer données EPR 2D complexes
print("🔬 Génération de données EPR 2D avec plusieurs signaux...")

# Axes 2D
field_2d = np.linspace(3300, 3400, 100)
angle_2d = np.linspace(0, 180, 80)
F_2d, A_2d = np.meshgrid(field_2d, angle_2d)

# Ligne de base 2D
baseline_2d_complex = (20 + 0.03 * (F_2d - 3350) + 0.01 * (A_2d - 90) + 
                      0.0001 * (F_2d - 3350)**2 + 0.0002 * (A_2d - 90)**2)

# Plusieurs signaux 2D avec dépendances angulaires
signals_2d = np.zeros_like(F_2d)

# Signal principal avec dépendance angulaire
for i, angle in enumerate(angle_2d):
    # Position dépendante de l'angle
    center_field = 3350 + 15 * np.cos(np.radians(angle))
    amplitude = 40 + 20 * np.cos(np.radians(2 * angle))
    linewidth = 8 + 3 * np.sin(np.radians(angle))
    
    signals_2d[i, :] = amplitude * linewidth**2 / ((field_2d - center_field)**2 + linewidth**2)

# Signal secondaire fixe
signal_fixed = 25 * 6**2 / ((F_2d - 3320)**2 + 6**2) * np.exp(-((A_2d - 45)**2 / 1000))
signals_2d += signal_fixed

# Données finales
noise_2d = np.random.normal(0, 2, F_2d.shape)
data_2d_complex = baseline_2d_complex + signals_2d + noise_2d

print(f"✅ Données 2D générées: {data_2d_complex.shape}")
print(f"📊 Signal principal: dépendance angulaire")
print(f"📊 Signal secondaire: fixe à 3320G, 45°")

# Affichage des données
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Données complètes
im1 = axes[0].pcolormesh(F_2d, A_2d, data_2d_complex, shading='auto', cmap='viridis')
axes[0].set_title('Données 2D Complètes\n(baseline + signaux)')
axes[0].set_xlabel('Champ Magnétique (G)')
axes[0].set_ylabel('Angle (deg)')
fig.colorbar(im1, ax=axes[0])

# Signaux seuls
im2 = axes[1].pcolormesh(F_2d, A_2d, signals_2d, shading='auto', cmap='plasma')
axes[1].set_title('Signaux EPR Seuls')
axes[1].set_xlabel('Champ Magnétique (G)')
fig.colorbar(im2, ax=axes[1])

# Baseline seule
im3 = axes[2].pcolormesh(F_2d, A_2d, baseline_2d_complex, shading='auto', cmap='coolwarm')
axes[2].set_title('Ligne de Base Seule')
axes[2].set_xlabel('Champ Magnétique (G)')
fig.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Définition manuelle de régions 2D à exclure
print("✂️ Définition de régions 2D à exclure...")

# Régions rectangulaires à exclure
exclude_regions_2d = [
    ((3340, 3370), (80, 100)),   # Signal principal central
    ((3315, 3325), (35, 55)),    # Signal secondaire  
    ((3330, 3350), (0, 20)),     # Signal aux petits angles
    ((3350, 3370), (160, 180)),  # Signal aux grands angles
]

print(f"📋 Régions 2D définies: {len(exclude_regions_2d)}")
for i, ((x1, x2), (y1, y2)) in enumerate(exclude_regions_2d):
    print(f"   Région {i+1}: Champ {x1}-{x2}G, Angle {y1}-{y2}°")

# Correction 2D avec régions manuelles
print("\n🔧 Correction 2D avec exclusion manuelle...")
corrected_2d_manual, baseline_2d_manual = epyr.baseline.baseline_polynomial_2d_simple(
    [field_2d, angle_2d], data_2d_complex, {},
    order=(2, 1),  # Ordre différent pour x et y
    exclude_center=False,
    manual_regions=exclude_regions_2d,
    region_mode='exclude'
)

print("✅ Correction 2D manuelle terminée!")

# Correction automatique pour comparaison
corrected_2d_auto, baseline_2d_auto = epyr.baseline.baseline_polynomial_2d_simple(
    [field_2d, angle_2d], data_2d_complex, {},
    order=(2, 1),
    exclude_center=True,
    center_fraction=0.4
)

# Visualisation des résultats
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Ligne 1: Baselines
im1 = axes[0,0].pcolormesh(F_2d, A_2d, baseline_2d_manual, shading='auto', cmap='plasma')
axes[0,0].set_title('Baseline Manuelle')
axes[0,0].set_xlabel('Champ Magnétique (G)')
axes[0,0].set_ylabel('Angle (deg)')
fig.colorbar(im1, ax=axes[0,0])

# Marquer régions exclues
for (x1, x2), (y1, y2) in exclude_regions_2d:
    from matplotlib.patches import Rectangle
    rect = Rectangle((x1, y1), x2-x1, y2-y1, 
                    linewidth=2, edgecolor='red', facecolor='none')
    axes[0,0].add_patch(rect)

im2 = axes[0,1].pcolormesh(F_2d, A_2d, baseline_2d_auto, shading='auto', cmap='plasma')
axes[0,1].set_title('Baseline Automatique')
axes[0,1].set_xlabel('Champ Magnétique (G)')
fig.colorbar(im2, ax=axes[0,1])

im3 = axes[0,2].pcolormesh(F_2d, A_2d, baseline_2d_complex, shading='auto', cmap='plasma')
axes[0,2].set_title('Baseline Vraie')
axes[0,2].set_xlabel('Champ Magnétique (G)')
fig.colorbar(im3, ax=axes[0,2])

# Ligne 2: Données corrigées
im4 = axes[1,0].pcolormesh(F_2d, A_2d, corrected_2d_manual, shading='auto', cmap='seismic')
axes[1,0].set_title('Corrigé (Manuel)')
axes[1,0].set_xlabel('Champ Magnétique (G)')
axes[1,0].set_ylabel('Angle (deg)')
fig.colorbar(im4, ax=axes[1,0])

im5 = axes[1,1].pcolormesh(F_2d, A_2d, corrected_2d_auto, shading='auto', cmap='seismic')
axes[1,1].set_title('Corrigé (Auto)')
axes[1,1].set_xlabel('Champ Magnétique (G)')
fig.colorbar(im5, ax=axes[1,1])

im6 = axes[1,2].pcolormesh(F_2d, A_2d, signals_2d, shading='auto', cmap='seismic')
axes[1,2].set_title('Signaux Vrais')
axes[1,2].set_xlabel('Champ Magnétique (G)')
fig.colorbar(im6, ax=axes[1,2])

plt.tight_layout()
plt.show()

# Calcul des erreurs
error_2d_manual = corrected_2d_manual - signals_2d
error_2d_auto = corrected_2d_auto - signals_2d

rms_2d_manual = np.sqrt(np.mean(error_2d_manual**2))
rms_2d_auto = np.sqrt(np.mean(error_2d_auto**2))

print(f"\n📊 Statistiques correction 2D:")
print(f"   RMS erreur manuelle: {rms_2d_manual:.4f}")
print(f"   RMS erreur automatique: {rms_2d_auto:.4f}")
print(f"   Amélioration: {rms_2d_auto/rms_2d_manual:.1f}x meilleure avec sélection manuelle!")

## Conclusion

### 🎯 **Nouvelles Capacités de Sélection de Régions:**

1. **Sélection Manuelle Programmable**:
   ```python
   # 1D
   regions = [(x1, x2), (x3, x4), ...]
   
   # 2D  
   regions = [((x1,x2), (y1,y2)), ((x3,x4), (y3,y4)), ...]
   ```

2. **Modes d'Utilisation**:
   - `region_mode='exclude'` - Exclure les régions spécifiées
   - `region_mode='include'` - Utiliser seulement les régions spécifiées

3. **Sélection Interactive**:
   ```python
   corrected, baseline = epyr.baseline.baseline_polynomial_1d_simple(
       x, y, params, interactive=True
   )
   ```

### 📊 **Avantages Démontrés:**
- **Précision améliorée** pour spectres complexes
- **Flexibilité maximale** - adaptable à tout type de signal
- **Interface intuitive** - sélection graphique simple
- **Compatible** avec les données `eprload()`

### 🔧 **Cas d'Usage Idéaux:**
- Spectres avec **plusieurs signaux**
- **Lignes de base non-uniformes**
- **Signaux à positions variables**
- **Données 2D complexes** (balayages angulaires, etc.)

**La sélection manuelle de régions révolutionne la correction de ligne de base EPR!** 🎉